Question: find the largest country (in terms ofpopulation) in each continent using the table below. 
Show the continent, the country name and the population in descending order (the largest country comesfirst)

Which country accounts for the majority of itscontinent's population?
-- (continent, country, population, perc_continent)
-- (Oceania, Australia, 24309330, 60.9235701)

In [7]:
import pandas as pd
import pandasql as pdsql

In [5]:
pop = pd.read_csv("population.csv")

In [10]:
pysql = lambda q: pdsql.sqldf(q, globals())
sql = """select bs.*
            from pop bs
            inner join
            (select continent, max(population) as max_pop
             from pop
             group by continent) max
            on bs.continent = max.continent and bs.population = max.max_pop
            order by population desc
        """
df = pysql(sql)

In [11]:
df

,Continent,Country,Population
0,Asia,China,1382323332
1,North America,US,324118787


In [18]:
# Python
max_pop = pop.sort_values(by = ["Continent","Population"], ascending = False).drop_duplicates(['Continent'], keep = 'first')
max_pop.sort_values(by = "Population", ascending = False, inplace = True)

In [19]:
max_pop

,Continent,Country,Population
3,Asia,China,1382323332
1,North America,US,324118787


In [51]:
sql2 = """ select
                 bs.continent
                 , bs.Country
                 , bs.Population
                 , sum.tot_pop
                 , bs.Population/ sum.tot_pop * 100 as perc_continent
            from pop bs
            inner join
            (select continent, sum(population) as tot_pop
                from pop
                group by continent) sum
            on bs.continent = sum.continent
            
        """
df2 = pysql(sql2)

In [52]:
df2

,Continent,Country,Population,tot_pop,perc_continent
0,North America,Canada,36286378,489037169,0
1,North America,US,324118787,489037169,0
2,North America,Mexico,128632004,489037169,0
3,Asia,China,1382323332,2803569108,0
4,Asia,India,1326801576,2803569108,0
5,Asia,Vietnam,94444200,2803569108,0


In [38]:
df2["perc_continent"] = df2.Population/ df2.tot_pop * 100

,Continent,Country,Population,tot_pop,perc_continent
0,North America,Canada,36286378,489037169,7.419963
1,North America,US,324118787,489037169,66.276923
2,North America,Mexico,128632004,489037169,26.303114
3,Asia,China,1382323332,2803569108,49.305841
4,Asia,India,1326801576,2803569108,47.325446
5,Asia,Vietnam,94444200,2803569108,3.368713


In [49]:
# Python
continent_pop = pop.groupby("Continent")["Population"].sum().to_frame().reset_index().rename(columns = {"Population": "total_pop"})
continent_top = pop.merge(continent_pop, left_on = "Continent", right_on = "Continent", how = "inner")
continent_top["perc_continent"] = continent_top.Population/ continent_top.total_pop * 100
continent_top.drop("total_pop", axis = 1).sort_values("perc_continent", ascending = False).head(1)

,Continent,Country,Population,perc_continent
1,North America,US,324118787,66.276923


In [47]:
continent_top

,Continent,Country,Population,total_Pop
0,North America,Canada,36286378,489037169
1,North America,US,324118787,489037169
2,North America,Mexico,128632004,489037169
3,Asia,China,1382323332,2803569108
4,Asia,India,1326801576,2803569108
5,Asia,Vietnam,94444200,2803569108
